# Simulation of countermeasure combinations

This notebook contains simulation of combinations of countermeasures and their behavior under our tests:
 - [GSR + Add](#GSR-+-Add)
 - [GSR + Mul](#GSR-+-Mul)
 - [GSR + Euclid](#GSR-+-Euclid)
 - [Add + Mul](#Add-+-Mul)
 - [Add + Euclid](#Add-+-Euclid)
 - [Mul + Euclid](#Mul-+-Euclid)

In [ ]:
import io
import math
import random
import itertools
import warnings

import cypari2
from cysignals.alarm import alarm, AlarmInterrupt

from matplotlib import pyplot as plt
from collections import Counter
from tqdm.auto import tqdm, trange

from pyecsca.misc.cfg import TemporaryConfig
from pyecsca.misc.utils import TaskExecutor
from pyecsca.ec.mod import mod, RandomModAction
from pyecsca.ec.point import Point
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.params import load_params_ectester, get_params
from pyecsca.ec.mult import LTRMultiplier, RTLMultiplier, ScalarMultiplicationAction
from pyecsca.ec.context import local, DefaultContext
from pyecsca.ec.formula import AdditionFormula
from pyecsca.ec.mod import Mod, mod
from pyecsca.ec.mult import ScalarMultiplier, ScalarMultiplicationAction
from pyecsca.ec.params import DomainParameters
from pyecsca.ec.point import Point
from pyecsca.ec.countermeasures import *
from public import public
from abc import ABC, abstractmethod
from typing import Optional
# %matplotlib ipympl
import secrets

In [ ]:
# copy from simulation notebook

def generate_scalars_mod3(rem, samples):
    scalars = []
    while True:
        scalar = random.randint(0, params3n.full_order)
        if scalar % 3 == rem:
            scalars.append(scalar)
        if len(scalars) == samples:
            break
    return scalars

def test_3n(countermeasure, scalars):
    ctr = Counter()
    for k in tqdm(scalars, leave=False):
        mult.init(params3n, point3n)
        kP = mult.multiply(k).to_affine()
        mult.init(params3n, point3n)
        knP = mult.multiply(k + params3n.full_order).to_affine()
        mult.init(params3n, point3n)
        k2nP = mult.multiply(k + 2 * params3n.full_order).to_affine()

        countermeasure.init(params3n, point3n)
        res = countermeasure.multiply(k)
        aff = res.to_affine()
        if aff.equals(kP):
            ctr["k"] += 1
        elif aff.equals(knP):
            ctr["k + 1n"] += 1
        elif aff.equals(k2nP):
            ctr["k + 2n"] += 1
        else:
            ctr[aff] += 1
            
    print(ctr)
    for name, count in sorted(ctr.items()):
        print(f"{name}:\t{count}")

def test_3n_fixed_scalar(countermeasure, samples):
    test_3n(countermeasure, [key3n for _ in range(samples)])

def test_3n_random_scalar(countermeasure, samples):
    test_3n(countermeasure, [random.randint(0, params3n.full_order) for _ in range(samples)])

def test_3n_random_scalar_projected(countermeasure, samples):
    print("k = 0 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(0, samples))
    print()
    print("k = 1 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(1, samples))
    print()
    print("k = 2 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(2, samples))


def test_composite(countermeasure, samples):
    G = composite.generator
    Gaff = G.to_affine()
    correct = 0
    errors = 0
    wrong = 0
    for _ in range(samples):
        k = random.randrange(0, composite.full_order)
        countermeasure.init(composite, G)
        try:
            res = countermeasure.multiply(k)
            res_aff = composite.curve.affine_multiply(Gaff, k)
        
            if res.equals_scaled(res_aff.to_model(composite.curve.coordinate_model, composite.curve)):
                correct += 1
            else:
                wrong += 1
        except Exception as e:
            errors += 1
    print(f"{errors} errors, {wrong} wrong results")


def test_k10(countermeasure, samples, k_range = None, zero_fails = True, plot=True):
    G = paramsk10.generator
    Gaff = G.to_affine()
    if k_range is None:
        ks = list(range(2, 21))
    else:
        ks = list(k_range)
    fails = []
    for k in ks:
        correct = 0
        failed = 0
        expected = paramsk10.curve.affine_multiply(Gaff, k).to_model(paramsk10.curve.coordinate_model, paramsk10.curve)
        for _ in range(samples):
            with local(DefaultContext()) as ctx:
                countermeasure.init(paramsk10, paramsk10.generator)
                res = countermeasure.multiply(k)
            smults = set()
            ctx.actions[0].walk(lambda action: smults.add(action.scalar) if isinstance(action, ScalarMultiplicationAction) else None)
            if 0 in smults and zero_fails:
                failed += 1
                continue
            try:
                if res.equals_scaled(expected):
                    correct += 1
                else:
                    failed += 1
            except:
                failed += 1
        print(f"k = {k}: failed in {failed} out of {samples}.")
        fails.append(failed / samples)
    if plot:
        fig, ax = plt.subplots()
        xs = list(range(len(ks)))
        ax.plot(xs, fails, label="Error rate")
        if any(k > 100 for k in ks):
            ax.set_xticks(xs, (f"2^{int(math.log2(k))}" for k in ks))
        else:
            ax.set_xticks(xs, ks)
        ax.set_ylim(-0.05, 1.05)
        ax.set_xlabel("k")
        ax.set_ylabel("Error rate")
        ax.legend()
        plt.show()
    return fails

Let's initialize some useful objects first.

In [ ]:
model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
ltr = LTRMultiplier(add, dbl, complete=False)
rtl = RTLMultiplier(add, dbl, complete=False)
mult = ltr

gsr = GroupScalarRandomization(mult)
gsr160 = GroupScalarRandomization(mult, 160)
asplit = AdditiveSplitting(mult)
msplit = MultiplicativeSplitting(mult)
esplit = EuclideanSplitting(mult,add)
bt = BrumleyTuveri(mult)

In the 3n test the target is given domain parameters of order $3n$ but claimed order $n$. The target is then given a point of order $3n$ for scalar multiplication and the results are observed.

In [ ]:
key3n = 0x20959f2b437de1e522baf6d814911938157390d3ea5118660b852ab0d5387006
params3n = load_params_ectester(io.BytesIO(b"0xc381bb0394f34b5ed061c9107b66974f4d0a8ec89b9fe73b98b6d1368c7d974d,0x5ca6c5ee0a10097af291a8f125303fb1a3e35e8100411902245d691e0e5cb497,0x385a5a8bb8af94721f6fd10b562606d9b9df931f7fd966e96859bb9bd7c05836,0x4616af1898b92cac0f902a9daee24bbae63571cead270467c6a7886ced421f5e,0x34e896bdb1337e0ae5960fa3389fb59c2c8d6c7dbfd9aac33a844f8f98e433ef,0x412b3e5686fbc3ca4575edb0292232702ae721a7d4a230cc170a5561aa70e00f,0x01"), "projective")
bits3n = params3n.full_order.bit_length()
point3n = Point(X=mod(0x4a48addb2e471767b7cd0f6f1d4c27fe46f4a828fc20f950bd1f72c939b36a84, params3n.curve.prime),
                Y=mod(0x13384d38c353f862832c0f067e46a3e510bb6803c20745dfb31929f4a18d890d, params3n.curve.prime),
                Z=mod(1, params3n.curve.prime), model=coords)

In [ ]:
paramsk10 = get_params("secg", "secp256r1", "projective")

In [ ]:
composite = load_params_ectester(io.BytesIO(b"0xc7a3ef9fa4ea63b537eedefc6bd52c3f35dc45be933d44270a1536c2ff9b6543,0x395f3675858362cbe7ac0d3e85708750aa42428368ae6ab1fda0d2a56255039b,0x61ca87695d4f6147b35975326eeee1a77f93226487315cd2419b4a1fe23f32d1,0x56e9a905d29f0f512cf709522bdd43a862d4e32c46268eec2f4c3fd9a70cb9d6,0xaf77a4ef604d33e3cf6c2ecaaa2913a5c51660e40365832ab98488950f3c348e,0xc7a3ef9fa4ea63b537eedefc6bd52c40f5e8e3bfe0f6dd05ac513edbcaa3cc47,0x01"), "projective")
print(f"prime:\t0x{composite.curve.prime:x}")
print(f"a:\t0x{composite.curve.parameters['a']:x}")
print(f"b:\t0x{composite.curve.parameters['b']:x}")
print(f"G:\t[0x{composite.generator.X:x},\n\t 0x{composite.generator.Y:x}]")
print(f"n:\t0x{composite.order:x}")

## GSR + Add

In [ ]:
@public
class GSR_Add_1(ScalarMultiplierCountermeasure):
    r"""
    GSR with Add, option 1.
    [k+rn-s]+[s]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits: int = 32, add = None):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits = rand_bits
        self.add = add
        self.r: int = None
        self.s: int = None

    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = int(Mod.random(1 << self.rand_bits)) if self.r is None else self.r
            s = int(Mod.random(order)) if self.s is None else self.s
            gsr_scalar = scalar + r * order
            m1 = self.mult.multiply(gsr_scalar-s)
            m2 = self.mult.multiply(s)
            if self.add is None:
                res = self.mult._add(m1, m2)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, m1, m2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        
@public
class GSR_Add_2(ScalarMultiplierCountermeasure):
    r"""
    GSR with Add, option 2.
    [k-s+r_1*n]+[s+r_2*n]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits: int = 32, add = None):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits = rand_bits
        self.add = add
        self.r1: int = None
        self.r2: int = None
        self.s: int = None

    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r1 = int(Mod.random(1 << self.rand_bits))  if self.r1 is None else self.r1
            r2 = int(Mod.random(1 << self.rand_bits))  if self.r2 is None else self.r2
            order = self.params.order
            s = int(Mod.random(order))  if self.s is None else self.s
            add_scalar = int(mod(scalar,order) - mod(s,order))
            
            m1 = self.mult.multiply(add_scalar+r1*order)
            m2 = self.mult.multiply(s+r2*order)
            if self.add is None:
                res = self.mult._add(m1, m2)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, m1, m2, **self.params.curve.parameters
                )[0]
            return action.exit(res)

In [ ]:
ga1 = GSR_Add_1(mult)
ga2 = GSR_Add_2(mult)

In [ ]:
test_3n_random_scalar(ga1, 1000)

In [ ]:
test_3n_random_scalar(ga2, 1000)

In [ ]:
test_3n_random_scalar_projected(ga1,1000)

In [ ]:
test_3n_random_scalar_projected(ga2, 1000)

In [ ]:
test_3n_fixed_scalar(ga1,1000)

In [ ]:
test_3n_fixed_scalar(ga2, 1000)

In [ ]:
test_composite(ga1, 1000)

In [ ]:
test_composite(ga2, 1000)

In [ ]:
test_k10(ga1, 100, k_range = range(8,11))

In [ ]:
test_k10(ga2, 100, k_range = range(8,11))

#### Experiments with fixed masks

In [ ]:
gsrbits = 32
ga1 = GSR_Add_1(mult)
ga1.r1 = int(Mod.random(1 << gsrbits))
ga1.r2 = int(Mod.random(1 << gsrbits))
ga2 = GSR_Add_2(mult)
ga2.r1 = int(Mod.random(1 << gsrbits))
ga2.r2 = int(Mod.random(1 << gsrbits))

In [ ]:
test_3n_random_scalar_projected(ga1,10)

In [ ]:
test_3n_random_scalar_projected(ga2,10)

In [ ]:
test_3n_random_scalar_projected(ga2,10)

In [ ]:
order = params3n.order
ga1 = GSR_Add_1(mult)
ga1.s = int(Mod.random(order))
ga2 = GSR_Add_2(mult)
ga2.s = int(Mod.random(order))

In [ ]:
test_3n_random_scalar_projected(ga1,10)

In [ ]:
test_3n_random_scalar_projected(ga2,10)

### GSR + Mul

In [ ]:
@public
class GSR_Mult_1(ScalarMultiplierCountermeasure):
    r"""
     GSR with Mult, option 1.
    [ks^(-1)+rn][s]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, rand_bits_mult: int = 32):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits_gsr = rand_bits_gsr
        self.rand_bits_mult = rand_bits_mult
        self.r: int = None
        self.s: int = None
    
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = int(Mod.random(1 << self.rand_bits_gsr))  if self.r is None else self.r
            s = int(Mod.random(1 << self.rand_bits_mult))  if self.s is None else self.s
            
            S = self.mult.multiply(s)
            self.mult.init(self.params, S)
            
            ks_inv = scalar * mod(s, order).inverse()
            ks_inv_gsr = int(ks_inv) + r*order
            return action.exit(self.mult.multiply(ks_inv_gsr))

@public
class GSR_Mult_2(ScalarMultiplierCountermeasure):
    r"""
     GSR with Mult, option 2.
    [ks^(-1)+r_1*n][s+r_2*n]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, rand_bits_mult: int = 32):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits_gsr = rand_bits_gsr
        self.rand_bits_mult = rand_bits_mult
        self.s: int = None
        self.r1: int = None
        self.r2: int = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r1 = int(Mod.random(1 << self.rand_bits_gsr))  if self.r1 is None else self.r1
            r2 = int(Mod.random(1 << self.rand_bits_gsr))  if self.r2 is None else self.r2
            s = int(Mod.random(1 << self.rand_bits_mult))  if self.s is None else self.s
            s_gsr = s + r2*order 
            
            S = self.mult.multiply(s_gsr)
            self.mult.init(self.params, S)
            
            ks_inv = scalar * mod(s, order).inverse()
            ks_inv_gsr = int(ks_inv) + r1*order
            return action.exit(self.mult.multiply(ks_inv_gsr))
        

@public
class GSR_Mult_3(ScalarMultiplierCountermeasure):
    r"""
     GSR with Mult, option 2.
    [ks^(-1)][s+r*n]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, rand_bits_mult: int = 32):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits_gsr = rand_bits_gsr
        self.rand_bits_mult = rand_bits_mult
        self.r: int = None
        self.s: int = None
    
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = int(Mod.random(1 << self.rand_bits_gsr)) if self.r is None else self.r
            s = int(Mod.random(1 << self.rand_bits_mult)) if self.s is None else self.s
            s_gsr = s + r*order 
            
            S = self.mult.multiply(s_gsr)
            self.mult.init(self.params, S)
            
            ks_inv = scalar * mod(s, order).inverse()
            return action.exit(self.mult.multiply(int(ks_inv)))

In [ ]:
gm1 = GSR_Mult_1(mult)
gm2 = GSR_Mult_2(mult)
gm3 = GSR_Mult_3(mult)

In [ ]:
test_3n_random_scalar(gm1, 1000)

In [ ]:
test_3n_random_scalar(gm2, 1000)

In [ ]:
test_3n_random_scalar(gm3, 1000)

In [ ]:
test_3n_random_scalar_projected(gm1,1000)

In [ ]:
test_3n_random_scalar_projected(gm2,1000)

In [ ]:
test_3n_random_scalar_projected(gm3,1000)

In [ ]:
test_3n_fixed_scalar(gm1,1000)

In [ ]:
test_3n_fixed_scalar(gm2,1000)

In [ ]:
test_3n_fixed_scalar(gm3,1000)

In [ ]:
test_k10(gm1, 100, k_range = range(8,11))

In [ ]:
test_k10(gm2, 100, k_range = range(8,11))

In [ ]:
test_k10(gm3, 100, k_range = range(8,11))

In [ ]:
test_composite(gm1, 1000)

In [ ]:
test_composite(gm2, 1000)

In [ ]:
test_composite(gm3, 1000)

#### Experiments with fixed masks

In [ ]:
r = int(Mod.random(1 << 32))
gm1 = GSR_Mult_1(mult)
gm1.r = r
gm2 = GSR_Mult_2(mult)
gm2.r = r
gm3 = GSR_Mult_3(mult)
gm3.r = r

In [ ]:
test_3n_random_scalar_projected(gm1,10)

In [ ]:
test_3n_random_scalar_projected(gm2,10)

In [ ]:
test_3n_random_scalar_projected(gm3,10)

In [ ]:
s = int(Mod.random(1 << 32))
gm1 = GSR_Mult_1(mult)
gm1.s = s
gm2 = GSR_Mult_2(mult)
gm2.s = s
gm3 = GSR_Mult_3(mult)
gm3.s = s

In [ ]:
test_3n_random_scalar_projected(gm1,10)

In [ ]:
test_3n_random_scalar_projected(gm2,10)

In [ ]:
test_3n_random_scalar_projected(gm3,10)

### GSR + Euclid

In [ ]:
@public
class GSR_Eucl_1(ScalarMultiplierCountermeasure):
    r"""
     GSR with Eucl, option 1.
     k+rn = k1+k2s
    [k1]+[k2][s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_gsr = rand_bits_gsr
        self.r: int = None
        self.s: int = None

    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )
        
    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            
            order = self.params.order
            r = int(Mod.random(1 << self.rand_bits_gsr))  if self.r is None else self.r
            gsr = scalar+r*order
            half_bits = gsr.bit_length() // 2
            s = int(Mod.random(1 << half_bits))  if self.s is None else self.s
            S = self.mult.multiply(s)
            k1 = gsr % s
            k2 = gsr // s
            T = self.mult.multiply(k1)
            self.mult.init(self.params, S)
            R = self.mult.multiply(k2)
            if self.add is None:
                res = self.mult._add(R, T)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, T, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        
        
@public
class GSR_Eucl_2(ScalarMultiplierCountermeasure):
    r"""
     GSR with Eucl, option 2.
     k = k1+k2s
    [k1]+[k2][s+r*n]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_gsr = rand_bits_gsr
        self.r: int = None
        self.s: int = None
        
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r = int(Mod.random(1 << self.rand_bits_gsr))  if self.r is None else self.r
            s = int(Mod.random(1 << half_bits))  if self.s is None else self.s
            S = self.mult.multiply(s+r*order)
            k1 = scalar % s
            k2 = scalar // s
            T = self.mult.multiply(k1)
            self.mult.init(self.params, S)
            R = self.mult.multiply(k2)
            if self.add is None:
                res = self.mult._add(R, T)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, T, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        
@public
class GSR_Eucl_3(ScalarMultiplierCountermeasure):
    r"""
     GSR with Eucl, option 3.
     k = k1+k2s
    [k1+r1*n]+[k2+r2*n][s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_gsr = rand_bits_gsr
        self.r1: int = None
        self.r2: int = None
        self.s: int = None
    
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r1 = int(Mod.random(1 << self.rand_bits_gsr))  if self.r1 is None else self.r1
            r2 = int(Mod.random(1 << self.rand_bits_gsr))  if self.r2 is None else self.r2
            s = int(Mod.random(1 << half_bits))  if self.s is None else self.s
            S = self.mult.multiply(s)
            k1 = scalar % s
            k2 = scalar // s
            T = self.mult.multiply(k1+r1*order)
            self.mult.init(self.params, S)
            R = self.mult.multiply(k2+r2*order)
            if self.add is None:
                res = self.mult._add(R, T)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, T, **self.params.curve.parameters
                )[0]
            return action.exit(res)
 
@public
class GSR_Eucl_4(ScalarMultiplierCountermeasure):
    r"""
     GSR with Eucl, option 4.
     k = k1+k2s
    [k1+r1*n]+[k2+r2*n][s+r3*n]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_gsr = rand_bits_gsr
        self.r1: int = None
        self.r2: int = None
        self.r3: int = None
        self.s: int = None
    
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r1 = int(Mod.random(1 << self.rand_bits_gsr))  if self.r1 is None else self.r1
            r2 = int(Mod.random(1 << self.rand_bits_gsr))  if self.r2 is None else self.r2
            r3 = int(Mod.random(1 << self.rand_bits_gsr))  if self.r3 is None else self.r3
            s = int(Mod.random(1 << half_bits))  if self.s is None else self.s
            S = self.mult.multiply(s+r3*order)
            k1 = scalar % s
            k2 = scalar // s
            T = self.mult.multiply(k1+r1*order)
            self.mult.init(self.params, S)
            R = self.mult.multiply(k2+r2*order)
            if self.add is None:
                res = self.mult._add(R, T)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, T, **self.params.curve.parameters
                )[0]
            return action.exit(res)

In [ ]:
ge1 = GSR_Eucl_1(mult)
ge2 = GSR_Eucl_2(mult)
ge3 = GSR_Eucl_3(mult)
ge4 = GSR_Eucl_4(mult)

In [ ]:
test_3n_random_scalar(ge1, 1000)

In [ ]:
test_3n_random_scalar(ge2, 1000)

In [ ]:
test_3n_random_scalar(ge3, 1000)

In [ ]:
test_3n_random_scalar(ge4, 1000)

In [ ]:
test_3n_random_scalar_projected(ge1,1000)

In [ ]:
test_3n_random_scalar_projected(ge2,1000)

In [ ]:
test_3n_random_scalar_projected(ge3,1000)

In [ ]:
test_3n_random_scalar_projected(ge4,1000)

In [ ]:
test_3n_fixed_scalar(ge1,1000)

In [ ]:
test_3n_fixed_scalar(ge2,1000)

In [ ]:
test_3n_fixed_scalar(ge3,1000)

In [ ]:
test_3n_fixed_scalar(ge4,1000)

In [ ]:
test_composite(ge1, 1000)

In [ ]:
test_composite(ge2, 1000)

In [ ]:
test_composite(ge3, 1000)

In [ ]:
test_composite(ge4, 1000)

In [ ]:
test_k10(ge1, 100, k_range = range(8,11))

In [ ]:
test_k10(ge2, 100, k_range = range(8,11))

In [ ]:
test_k10(ge3, 100, k_range = range(8,11))

In [ ]:
test_k10(ge4, 100, k_range = range(8,11))

#### Experiments with fixed masks

In [ ]:
half_bits = params3n.order.bit_length() // 2
s = int(Mod.random(1 << half_bits))
ge1 = GSR_Eucl_1(mult)
ge1.s = s
ge2 = GSR_Eucl_2(mult)
ge2.s = s
ge3 = GSR_Eucl_3(mult)
ge3.s = s

In [ ]:
test_3n_random_scalar_projected(ge1,10)

In [ ]:
test_3n_random_scalar_projected(ge2,10)

In [ ]:
test_3n_random_scalar_projected(ge3,10)

In [ ]:
bits = 32
r1 = int(Mod.random(1 << bits))
r2 = int(Mod.random(1 << bits))
r3 = int(Mod.random(1 << bits))

ge1 = GSR_Eucl_1(mult)
ge1.r1 = r1
ge1.r2 = r2
ge1.r3 = r3
ge2 = GSR_Eucl_2(mult)
ge2.r1 = r1
ge2.r2 = r2
ge2.r3 = r3
ge3 = GSR_Eucl_3(mult)
ge3.r1 = r1
ge3.r2 = r2
ge3.r3 = r3

In [ ]:
test_3n_random_scalar_projected(ge1,10)

In [ ]:
test_3n_random_scalar_projected(ge2,10)

In [ ]:
test_3n_random_scalar_projected(ge3,10)

### Add + Mul

In [ ]:
@public
class Add_Mul_1(ScalarMultiplierCountermeasure):
    r"""
     Add with Mul, option 1.
    [(k-r)s1^(-1)][s1]+[rs2^(-1)][s2]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier,rand_bits_mul=32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_mul = rand_bits_mul
        self.r: int = None
        self.s1: int = None
        self.s2: int = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = int(Mod.random(order)) if self.r is None else self.r
            s1 = int(Mod.random(1 << self.rand_bits_mul)) if self.s1 is None else self.s1
            s2 = int(Mod.random(1 << self.rand_bits_mul)) if self.s2 is None else self.s2
            kr = int(mod(scalar,order) - mod(r,order))
            
            kr_inv = int(kr * mod(s1, order).inverse())
            r_inv = int(r * mod(s2, order).inverse())
            
            R = self.mult.multiply(s1)
            self.mult.init(self.params, R)
            R = self.mult.multiply(kr_inv)
            
            self.mult.init(self.params, self.point)
            S = self.mult.multiply(s2)
            self.mult.init(self.params, S)
            S = self.mult.multiply(r_inv)
            if self.add is None:
                res = self.mult._add(R, S)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, S, **self.params.curve.parameters
                )[0]
            return action.exit(res)

@public
class Add_Mul_2(ScalarMultiplierCountermeasure):
    r"""
     Add with Mul, option 2.
    ([ks^(-1)-r]+[r])[s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier,rand_bits_mul=32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_mul = rand_bits_mul
        self.r = None
        self.s = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = Mod.random(order) if self.r is None else self.r
            s = Mod.random(1 << self.rand_bits_mul) if self.s is None else self.s
            
            ks_inv = scalar * mod(int(s), order).inverse()
            
            S = self.mult.multiply(int(s))
            self.mult.init(self.params, S)
            
            R = self.mult.multiply(int(ks_inv-r))
            S = self.mult.multiply(int(r))

            if self.add is None:
                res = self.mult._add(R, S)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, S, **self.params.curve.parameters
                )[0]
            return action.exit(res)



In [ ]:
am1 = Add_Mul_1(mult)
am2 = Add_Mul_2(mult)

In [ ]:
test_3n_random_scalar(am1, 1000)

In [ ]:
test_3n_random_scalar(am2, 1000)

In [ ]:
test_3n_random_scalar_projected(am1, 1000)

In [ ]:
test_3n_random_scalar_projected(am2, 1000)

In [ ]:
test_3n_fixed_scalar(am1,1000)

In [ ]:
test_3n_fixed_scalar(am2,1000)

In [ ]:
test_composite(am1, 1000)

In [ ]:
test_composite(am2, 1000)

In [ ]:
test_k10(am1, 100, k_range = range(8,11))

In [ ]:
test_k10(am2, 100, k_range = range(8,11))

### Add + Euclid

In [ ]:
      
@public
class Add_Eucl_1(ScalarMultiplierCountermeasure):
    r"""
     Add with Eucl, option 1.
     k-r = k1+k2s1
     r = l1+l2s2
    [k1]+[k2][s1]+[l1]+[l2][s2]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.r: int = None
        self.s1: int = None
        self.s2: int = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r = int(Mod.random(order)) if self.r is None else self.r
            s1 = int(Mod.random(1 << half_bits)) if self.s1 is None else self.s1
            s2 = int(Mod.random(1 << half_bits)) if self.s2 is None else self.s2
            k_r = int(mod(scalar,order)-mod(r,order))
            k1 = k_r % s1
            k2 = k_r // s1
            
            l1 = r % s2
            l2 = r // s2
            
            
            S1 = self.mult.multiply(s1)
            S2 = self.mult.multiply(s2)
            K1 = self.mult.multiply(k1)
            L1 = self.mult.multiply(l1)
            
            self.mult.init(self.params, S1)
            K2 = self.mult.multiply(k2)
            self.mult.init(self.params, S2)
            L2 = self.mult.multiply(l2)
            
            if self.add is None:
                res = self.mult._add(K1, K2)
                res = self.mult._add(res, L1)
                res = self.mult._add(res, L2)
            else:
                res = self.add(
                    self.params.curve.prime, K1, K2, **self.params.curve.parameters
                )[0]
                res = self.add(
                    self.params.curve.prime, res, L1, **self.params.curve.parameters
                )[0]
                res = self.add(
                    self.params.curve.prime, res, L2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        

          
@public
class Add_Eucl_2(ScalarMultiplierCountermeasure):
    r"""
     Add with Eucl, option 2.
     k = k1+k2s
    [k1-r1]+[r1]+([k2-r2]+[r2])[s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.r1: int = None
        self.r2: int = None
        self.s: int = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r1 = int(Mod.random(order)) if self.r1 is None else self.r1
            r2 = int(Mod.random(order)) if self.r2 is None else self.r2
            s = int(Mod.random(1 << half_bits)) if self.s is None else self.s
            k1 = scalar % s
            k2 = scalar // s
            k1r1 = int(mod(k1,order)-mod(r1,order))
            k2r2 = int(mod(k2,order)-mod(r2,order))
            
            
            S = self.mult.multiply(s)
            KR1 = self.mult.multiply(k1r1)
            R1 = self.mult.multiply(r1)
            
            self.mult.init(self.params, S)
            KR2 = self.mult.multiply(k2r2)
            R2 = self.mult.multiply(r2)
            
            if self.add is None:
                res = self.mult._add(KR1, R1)
                res = self.mult._add(res, KR2)
                res = self.mult._add(res, R2)
            else:
                res = self.add(
                    self.params.curve.prime, KR1, R1, **self.params.curve.parameters
                )[0]
                res = self.add(
                    self.params.curve.prime, res, KR2, **self.params.curve.parameters
                )[0]
                res = self.add(
                    self.params.curve.prime, res, R2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        


In [ ]:
ae1 = Add_Eucl_1(mult)
ae2 = Add_Eucl_2(mult)

In [ ]:
test_3n_random_scalar(ae1, 1000)

In [ ]:
test_3n_random_scalar(ae2, 1000)

In [ ]:
test_3n_random_scalar_projected(ae1, 1000)

In [ ]:
test_3n_random_scalar_projected(ae2, 1000)

In [ ]:
test_3n_fixed_scalar(ae1,1000)

In [ ]:
test_3n_fixed_scalar(ae2,1000)

In [ ]:
test_composite(ae1, 1000)

In [ ]:
test_composite(ae2, 1000)

In [ ]:
test_k10(ae1, 100, k_range = range(8,11))

In [ ]:
test_k10(ae2, 100, k_range = range(8,11))

### Mul + Euclid

In [ ]:
@public
class Mul_Eucl_1(ScalarMultiplierCountermeasure):
    r"""
     Mul with Eucl, option 1.
     kr^(-1) = k1+k2s
    ([k1]+[k2][s])[r]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier,rand_bits_mul=32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_mul = rand_bits_mul
        self.r: int = None
        self.s: int = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            s = int(Mod.random(order)) if self.s is None else self.s
            r = int(Mod.random(1 << self.rand_bits_mul)) if self.r is None else self.r
            
            kr_inv = int(scalar * mod(int(r), order).inverse())
            
            k1 = kr_inv % s
            k2 = kr_inv // s
            
            R = self.mult.multiply(r)
            self.mult.init(self.params, R)
            
            K1 = self.mult.multiply(k1)
            
            S = self.mult.multiply(s)
            self.mult.init(self.params, S)
            K2 = self.mult.multiply(k2)
            
            if self.add is None:
                res = self.mult._add(K1, K2)
            else:
                res = self.add(
                    self.params.curve.prime, K1, K2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        

                 
@public
class Mul_Eucl_2(ScalarMultiplierCountermeasure):
    r"""
     Mul with Eucl, option 2.
    [k1r1^(-1)][r1]+[k2r2^(-1)][r2][s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier,rand_bits_mul=32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_mul = rand_bits_mul
        self.s: int = None
        self.r1: int = None
        self.r2: int = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            s = int(Mod.random(order)) if self.s is None else self.s
            
            r1 = int(Mod.random(1 << self.rand_bits_mul)) if self.r1 is None else self.r1
            r2 = int(Mod.random(1 << self.rand_bits_mul)) if self.r2 is None else self.r2
            
            k1 = scalar % s
            k2 = scalar // s
            
            kr1_inv = int(k1 * mod(int(r1), order).inverse())
            kr2_inv = int(k2 * mod(int(r2), order).inverse())
            
            R1 = self.mult.multiply(r1)
            S = self.mult.multiply(s)
            
            self.mult.init(self.params, R1)
            R1 = self.mult.multiply(kr1_inv)
            
            self.mult.init(self.params, S)
            R2 = self.mult.multiply(r2)
            
            self.mult.init(self.params, R2)
            R2 = self.mult.multiply(kr2_inv)
            
            
            if self.add is None:
                res = self.mult._add(R1, R2)
            else:
                res = self.add(
                    self.params.curve.prime, R1, R2, **self.params.curve.parameters
                )[0]
            return action.exit(res)



In [ ]:
me1 = Mul_Eucl_1(mult)
me2 = Mul_Eucl_2(mult)

In [ ]:
test_3n_random_scalar(me1, 1000)

In [ ]:
test_3n_random_scalar(me2, 1000)

In [ ]:
test_3n_random_scalar(me3, 1000)

In [ ]:
test_3n_random_scalar_projected(me1, 1000)

In [ ]:
test_3n_random_scalar_projected(me2, 1000)

In [ ]:
test_3n_fixed_scalar(me1,1000)

In [ ]:
test_3n_fixed_scalar(me2,1000)

In [ ]:
test_composite(me1, 1000)

In [ ]:
test_composite(me2, 1000)

In [ ]:
test_k10(me1, 100, k_range = range(8,11))

In [ ]:
test_k10(me2, 100, k_range = range(8,11))